## Akaike Internship Assignment - Phalgun Natarajan


#### Importing the addresses of all the numpy files

We have 98 files

In [1]:
import glob, os
import math
import numpy as np
import statistics as stat
from PIL import Image
import pandas as pd
path = r"c:\Users\Phalgun\Desktop\Data_Analytics\regions"
os.chdir(path)
files=[]
for file in glob.glob("*.npy"):
    files.append(file)
    
len(files)

98

This allows us to open and view an image

In [2]:
img = np.load(files[0])
img_rgb = Image.fromarray(np.uint8(img[:,:,:3], mode='RGB'))
img_rgb.show()

# Algo to fix parameters for every image
### Finding the average Greyscale value of every image to correlate it. Used Luminosity algorithm.

The 2 parameters used to characterize every image is :
- Greyscale (Luminosity) : This is a function of the RGB values. Luminosity = 0.2126*R + 0.7152*G + 0.0722*B
- Calculate the difference in the Max and min elevation in a picture. This is the differentiating factor between a roof and road. the comparison of differences of height is done, as we cannot directly compare the mean sea levels f 2 different places.

A provision has been made to ignore all empty pixels

The list 'parameters' is a list of the greyscale and Elevation difference values

In [3]:
parameters =[]
readings = 0

for img_no in range (len(files)):
    img = np.load(files[img_no])
    img_rgb = Image.fromarray(np.uint8(img[:,:,:3], mode='RGB'))
    x = img.shape[0]
    y = img.shape[1]
    total_lum = 0
    elv= []
    count = 0
    for i in range (x):
        for j in range(y):
            R = img[i][j][0]
            G = img[i][j][1]
            B = img[i][j][2]
            E = img[i][j][3]
            a = math.isnan(R)
            if (a ==False):
                lum = 0.2126*R + 0.7152*G + 0.0722*B
                total_lum = total_lum + lum
                elv.append(E)
                count = count+1
    greyscale = total_lum/count
    elv.sort()
    x= 0.2*count
    x=int(x)
    elv_diff = (sum(elv[-x:])-sum(elv[0:x]))/x
    parameters.append((greyscale,elv_diff)) 
    readings = readings + 1
    if (readings%5 == 0):
        print (readings, "out of 98 done, please wait")

5 out of 98 done, please wait
10 out of 98 done, please wait
15 out of 98 done, please wait
20 out of 98 done, please wait
25 out of 98 done, please wait
30 out of 98 done, please wait
35 out of 98 done, please wait
40 out of 98 done, please wait
45 out of 98 done, please wait
50 out of 98 done, please wait
55 out of 98 done, please wait


C:\Users\Phalgun\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: RuntimeWarning: divide by zero encountered in double_scalars


60 out of 98 done, please wait
65 out of 98 done, please wait
70 out of 98 done, please wait
75 out of 98 done, please wait
80 out of 98 done, please wait
85 out of 98 done, please wait
90 out of 98 done, please wait
95 out of 98 done, please wait


There seemed to be an error with the 58th image, and it was unable to load on my machine. Hence I have deleted to avoid error in our calculations

In [4]:
len(parameters)

98

In [5]:
del parameters[57]
parameters


[(165.26458305651403, 3.395170514611968),
 (91.36762059531874, 1.1443404910638804),
 (127.14779055645721, 1.0888708783693963),
 (143.14585482825547, 0.6192993699064238),
 (183.8206201038381, 1.1585886720207854),
 (150.418233967963, 0.2205700268508607),
 (104.92003499753862, 3.1127172082237045),
 (197.83978728313707, 7.62890773141051),
 (74.76653539249413, 2.2338088555734794),
 (193.27889376496717, 0.8007829439823497),
 (60.42607237974772, 14.841494543908341),
 (168.06342425929313, 1.330975237858703),
 (132.71227183160266, 1.9817421673334132),
 (169.1231012966855, 9.740618818624775),
 (202.98156503007337, 9.141698936547082),
 (214.39976091580144, 12.69841692456188),
 (63.4141049318102, 22.236498738201117),
 (188.68639021125807, 16.556975897841568),
 (171.5317235218122, 1.6572770723001422),
 (121.72112138148704, 2.7040068462417577),
 (111.73875735348811, 3.2470662283631504),
 (145.3488743643711, 0.9534209292141073),
 (92.84586312601927, 6.464253200759282),
 (76.26191108202298, 24.0557709

In [6]:
len(parameters)

97

### We will now normalize the data, to avoid giving too much weightage to Greyscale, over elevation value, using min max normalization.

In [7]:
a = max(parameters)[0]
b = max(parameters, key=lambda x: x[1])[1]
c = min(parameters)[0]
d = min(parameters, key=lambda x: x[1])[1]
print(a,b,c,d)
std_params = []
for i in range (len(parameters)):
    x = (parameters[i][0]-c)/(a-c)
    y = (parameters[i][1]-d)/(b-d)
    std_params.append((x,y))
std_params[0:4]

216.34707236621665 38.350452645737136 60.42607237974772 0.2196723080306389


[(0.6723822364265516, 0.0832791298383465),
 (0.19844375176054652, 0.02424991502518143),
 (0.42792002477215835, 0.022795194922334977),
 (0.5305236783735757, 0.010480432300007364)]

In [8]:
len(std_params)

97

## KNN Classification

 - Although not required for predicting the similarity, KNN will tell me how well my parameters work when it comes to predicting the class of new data.
 
 - The Test set accuracy is high, indicating that the parameters chosen are valid, and reflect the class of the data accurately

#### The labels are being read into the pandas dataframe, to be converted into a list

In [9]:
import pandas as pd

df = pd.read_csv(r'C:\Users\Phalgun\Desktop\Data_Analytics\ground_truth.csv') # can also index sheet by name or fetch all sheets
img_type = df['Type'].tolist()


This element was deleted in the previous lists, so we must delete it here

In [10]:
del img_type[57] # We delete this element as we have deleted it 

#### I have split the data set into a training and testing data set, with a test size of 60%.

Generally I would give a test size of just 20 - 30 %, but due to the small data set size, changed it to 60 %

In [11]:
from sklearn.model_selection import train_test_split
X= np.asarray(std_params)
Y = np.asarray(img_type)
X_train, X_test, y_train, y_test = train_test_split( X,Y, test_size=0.6, random_state=4)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

Train set: (38, 2) (38,)
Test set: (59, 2) (59,)


Using the KNN classifier with a K value of 4, meaning, we search for the 4 closest neighbours. Minkowski distance is used for calculating the distance

In [12]:
from sklearn.neighbors import KNeighborsClassifier
k = 4
#Train Model and Predict  
neigh = KNeighborsClassifier(n_neighbors = k).fit(X_train,y_train)
yhat = neigh.predict(X_test)
yhat[0:5]

array(['b', 'b', 'b', 'v', 'r'], dtype='<U1')

The similarity is checked by the metrics function in the sklearn library

In [13]:
from sklearn import metrics
print("Train set Accuracy: ", metrics.accuracy_score(y_train, neigh.predict(X_train)))
print("Test set Accuracy: ", metrics.accuracy_score(y_test, yhat))

Train set Accuracy:  0.8947368421052632
Test set Accuracy:  0.8813559322033898


# Clustering

I have performed a basic K-means clustering algorithm, in the case that we do not have the labelled data given to us. The number of centroids are 3, as given by the images.

In [14]:
from sklearn.cluster import KMeans 
clusterNum = 3
k_means = KMeans(init = "k-means++", n_clusters = clusterNum, n_init = 12)
k_means.fit(std_params)
labels = k_means.labels_
print(labels[0:20])

[1 0 0 0 1 0 0 1 0 1 2 1 0 1 1 1 2 1 1 0]


### Finding Intracluster distance and Intercluster distance for the clusters found here

Here I have attached the predicted cluster with the adjacent parameters, ie. greyscale and elevation

In [15]:
predicted_cluster_list=[]
for i in range ((len(labels)-1)):
    z=labels[i]
    x=std_params[i][0]
    y=std_params[i][1]
    predicted_cluster_list.append((x,y,z))
predicted_cluster_list[0:5]          
len(predicted_cluster_list)

96

### Finding the new centroids by average of location

In [16]:
gs_total0 = 0
e_total0 = 0
gs_total1 = 0
e_total1= 0
gs_total2 = 0
e_total2= 0
count0=0
count1=0
count2=0
for i in range (len(predicted_cluster_list)-1):
    if (predicted_cluster_list[i][2]==0):
        x0 = predicted_cluster_list[i][0]
        y0= predicted_cluster_list [i][1]
        gs_total0 = gs_total0 + x0
        e_total0 = e_total0 + y0
        count0 = count0 + 1
    if (predicted_cluster_list[i][2]==1):
        x1 = predicted_cluster_list[i][0]
        y1 = predicted_cluster_list [i][1]
        gs_total1 = gs_total1 + x1
        e_total1 = e_total1 + y1
        count1 = count1 + 1
    if (predicted_cluster_list[i][2]==2):
        x2 = predicted_cluster_list[i][0]
        y2 = predicted_cluster_list [i][1]
        gs_total2 = gs_total2 + x2
        e_total2 = e_total2 + y2
        count2 = count2 + 1
centroid1 = [gs_total1/count1,e_total1/count1]
centroid0 = [gs_total0/count1,e_total0/count0]
centroid2 = [gs_total2/count2,e_total2/count2]
print(centroid0,centroid1,centroid2)
        

[0.7765599377925657, 0.09323812109678128] [0.7697699125559624, 0.22990984103932674] [0.1848670810716692, 0.5063404875042757]


### Finding the intracluster euclidean distances

In [17]:
ed0=0
ed1=0
ed2=0
for i in range (len(predicted_cluster_list)-1):
    if (predicted_cluster_list[i][2]==0):
        x0 = predicted_cluster_list[i][0]
        y0= predicted_cluster_list [i][1]
        z0= centroid0[0]
        c0= centroid0[1]
        ed = ((x0-z0)**2 + (y0-c0)**2)**.5
        ed0 = ed0 + ed
        count0 = count0 + 1
    if (predicted_cluster_list[i][2]==1):
        x1 = predicted_cluster_list[i][0]
        y1 = predicted_cluster_list [i][1]
        z1= centroid1[0]
        c1= centroid1[1]
        ed = ((x1-z1)**2 + (y1-c1)**2)**.5
        ed1 = ed1 + ed
        count1 = count1 + 1
    if (predicted_cluster_list[i][2]==2):
        x2 = predicted_cluster_list[i][0]
        y2 = predicted_cluster_list [i][1]
        z2= centroid2[0]
        c2= centroid2[1]
        ed = ((x2-z2)**2 + (y2-c2)**2)**.5
        ed2 = ed2 + ed
        count2 = count2 + 1
euclid_dist0=ed0/count0
euclid_dist1=ed1/count1
euclid_dist2=ed2/count2
print(euclid_dist0,euclid_dist1,euclid_dist2)

0.18455438845465907 0.10151164606636622 0.07274767229814959


### Finding the intercluster euclidean distances

In [18]:
c0toc1 = ((centroid0[0]-centroid1[0])**2 + (centroid0[1]-centroid1[1])**2)**0.5
c0toc2 = ((centroid0[0]-centroid2[0])**2 + (centroid0[1]-centroid2[1])**2)**0.5
c1toc2 = ((centroid1[0]-centroid2[0])**2 + (centroid1[1]-centroid2[1])**2)**0.5

print(c0toc1,c1toc2,c0toc2)

0.13684028454650074 0.6469352553257344 0.7216328719134179


### The effectiveness of the model

- Effectiveness is given by (compactness) / (separation)

In [19]:
effectiveness=1-(euclid_dist0+euclid_dist1+euclid_dist2)/(c0toc1+c1toc2+c0toc2)
effectiveness

0.7616502578236792

Clearly, we can see that the similarity i.e. the euclidean distance, is much higher within the cluster, as compared to the distance between clusters.

# Similarity calculation

### Similarity of the K-means clustering algorithm

Here I have found the euclidean distance between any 2 images. By entering which 2 images to compare, the distances between is calculated. 

- Distance = Euclidean distance
- Similarity = 1 - distance
- The classes of the image are shown below, as you can see, the images of same class have better similarity

In [20]:
pic1,pic2 = input("Which 2 images do you want to check ? (Enter a value b/w 1-98, separated by space)").split()
print("We will check similarity between", pic1, "and", pic2)

pic1 = int(pic1) - 1
pic2 = int(pic2) -1
cluster_dist = ((predicted_cluster_list[pic1][0]-predicted_cluster_list[pic2][0])**2 + (predicted_cluster_list[pic1][1]-predicted_cluster_list[pic2][1])**2)**0.5
cluster_sim = 1- cluster_dist
print( "Image",pic1+1,"has properties",predicted_cluster_list[pic1],"and belongs to class",predicted_cluster_list[pic1][2])
print("Image",pic2+1,"has properties",predicted_cluster_list[pic2],"and belongs to class",predicted_cluster_list[pic2][2])
print("The similarity between these images is",cluster_sim)

Which 2 images do you want to check ? (Enter a value b/w 1-98, separated by space)2 56
We will check similarity between 2 and 56
Image 2 has properties (0.19844375176054652, 0.02424991502518143, 0) and belongs to class 0
Image 56 has properties (0.3595505222386182, 0.16392036808005628, 0) and belongs to class 0
The similarity between these images is 0.7867789247038931


## Accuracy of the clustering model in comparison of the actual model

- This was calculated by comparing the actual values of every image to the predicted value
- Accuracy = Correct_prediction / Total_records

In [21]:
n_labels=[]
for i in range (len(img_type)-1):
    x = img_type[i]
    if (x == 'v'):
        y = labels[10]
    if (x == 'r'):
        y = labels[1]
    if (x=='b'):
        y= labels[0]
    n_labels.append(y)

In [22]:
correct = 0
for i in range (len(labels)-1):
    if (labels[i]==n_labels[i]):
        correct = correct + 1
accuracy = correct/(len(labels))
print ("Accuracy of the k-means algorithm is",accuracy)

Accuracy of the k-means algorithm is 0.7628865979381443


### Algo Similarity
This is the similarity based on the greyscale and Elevation properties allocated to each image, in comparison to the actual labels of each image.

- A list was made comprising of the parameters I assigned to the image, and their corresponding actual labels
- You can check the similarity of 2 images, and notice the classes they are a part of too

In [24]:
pic1,pic2 = input("Which 2 images do you want to check ? (Enter a value b/w 1-98, separated by space)").split()
print("We will check similarity between", pic1, "and", pic2)

pic1 = int(pic1) - 1
pic2 = int(pic2) -1
masterlist=[]
for i in range ((len(std_params)-1)):
    z=img_type[i]
    x=std_params[i][0]
    y=std_params[i][1]
    masterlist.append((x,y,z))
euclid_dist = ((masterlist[pic1][0]-masterlist[pic2][0])**2 + (masterlist[pic1][1]-masterlist[pic2][1])**2)**0.5
similarity = 1- euclid_dist
print( "Image",pic1+1,"has properties",masterlist[pic1],"and belongs to class",masterlist[pic1][2])
print("Image",pic2+1,"has properties",masterlist[pic2],"and belongs to class",masterlist[pic2][2])
print("The similarity between these images is",similarity)

Which 2 images do you want to check ? (Enter a value b/w 1-98, separated by space)2 65
We will check similarity between 2 and 65
Image 2 has properties (0.19844375176054652, 0.02424991502518143, 'r') and belongs to class r
Image 65 has properties (0.280738055884955, 0.015867699378055072, 'r') and belongs to class r
The similarity between these images is 0.9172799055218409
